## Руководитель агентов

В [предыдущем примере](multi-agent-collaboration.ipynb) вы можете ознакомиться с автоматической маршрутизацией сообщений в зависимости от результата работы первичного агента-исследователя.

Кроме такого подхода вы можете использовать LLM для организации работы агентов.

Пример ниже показывает как создать группу агентов с LLM в роли руководителя, который помогает распределять задачи.

![Схема](./img/supervisor-diagram.png)

Для упрощения кода каждой из вершин, в которых распологаются агенты, используется класс `AgentExecutor` библиотеки LangChain. Таким образом этот и другие сложные примеры показывают различные подходы к проектированию с помощью LangGraph. Комбинируйте эти подходы с другими базовыми решениями, для более эффективного решения своих задач. 

Перед началом разработки подготовьте среду:

In [1]:
# %%capture --no-stderr
# %pip install -U langchain langchain_openai langchain_experimental langsmith pandas

In [1]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

## Создание инструментов

Для этого примера вам потребуется два агента. Первый будет выполнять исследование с помощью запросов к поисковой системе, а второй будет отрисовывать графики.

Задайте инструменты, которые будут использовать каждый из агентов:

In [ ]:
from typing import Annotated, List, Tuple, Union

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)

# Функция выполняет код локально, что может быть небезсопасно
python_repl_tool = PythonREPLTool()

## Создание вспомогательных функций

Создайте вспомогательную функцию, которая упростит добавление вершин с агентами-исполнителями:

In [3]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Каждой вершине с исполнителем присваивается имя и дается доступ
    # к определенным инструментам
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

Также определите функцию, которая будет представлять вершины графа. Функция будет преобразовывать ответ агента в сообщения от человека (`HumanMessage`). Это нужно для работы с глобальным состоянием графа.

In [4]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

### Создайте руководителя агентов

Руководитель вызывает функцию, чтобы определенить следующую вершину-исполнителя или завершить обработку.

In [5]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)
# Руководитель агентов — это LLM-вершина графа. Руководитель определяет
# какому из агентов передать задачу и принимает решение о ее выполнении
options = ["FINISH"] + members
# Использование openai-функций упрощает обработку вывода
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model="gpt-4-1106-preview")

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

## Соберите граф

Теперь вы можете собрать граф.
Для этого определите состояние агента и вершины-исполнители. Для определения вершин используйте созданную вспомогательную функцию.

In [6]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# Состояние агента передается на вход каждой из вершин графа
class AgentState(TypedDict):
    # Аннотация указывает графу, что новые сообщения нужно всегда
    # добавлять к текущим состояниям
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # Поле 'next' указывает, что делать дальше
    next: str


research_agent = create_agent(llm, [tavily_tool], "You are a web researcher.")
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# NOTE: НА ЭТОМ ЭТАПЕ ВЫПОЛНЯЕТСЯ ПРОИЗВОЛЬНЫЙ КОД. БУДЬТЕ ОСТОРОЖНЫ
code_agent = create_agent(
    llm,
    [python_repl_tool],
    "You may generate safe python code to analyze data and generate charts using matplotlib.",
)
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)

Соедините все ребра графа.

In [7]:
for member in members:
    # Исполнители должны ВСЕГДА докладывать руководителю о завершении работы
    workflow.add_edge(member, "supervisor")
# Руководитель заполняет поле "next" в состостоянии графа,
# которое переходит к другой вершине или заканчивает обработку
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Добавьте точку входа
workflow.set_entry_point("supervisor")

graph = workflow.compile()

## Запустите граф

Теперь, когда граф собран, вы можете запустить его и посмотреть, как он работает.

In [8]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Code hello world and print it to the terminal")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Coder'}}
----


Python REPL can execute arbitrary code. Use with caution.


{'Coder': {'messages': [HumanMessage(content="The code `print('Hello, World!')` was executed, and the output is:\n\n```\nHello, World!\n```", name='Coder')]}}
----
{'supervisor': {'next': 'FINISH'}}
----


In [9]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Write a brief research report on pikas.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='**Research Report on Pikas**\n\nPikas are small mammals related to rabbits, known for their distinctive chirping sounds. They inhabit some of the most challenging environments, particularly boulder fields at high elevations, such as those found along the treeless slopes of the Southern Rockies, where they can be found at altitudes of up to 14,000 feet. Pikas are well-adapted to cold climates and typically do not fare well in warmer temperatures.\n\nRecent studies have shown that pikas are being impacted by climate change. Research by Peter Billman, a Ph.D. student from the University of Connecticut, indicates that pikas have moved upslope by approximately 1,160 feet. This upslope retreat is a direct response to changing climatic conditions, as pikas seek cooler temperatures at higher elevations.\n\nPikas are also known to be industrious foragers, particularly during the summer months when the